In [3]:
import numpy as np
import torch
import os
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import time

from NS_data import create_residual_data, create_ICBC_data
from NS_GPT_activation import P
from NS_GPT_precomp import autograd_calculations, Pt_nu_lap_vor
from NS_GPT_PINN import GPT
from NS_GPT_train import gpt_train

plt.rc('font', size=10)

torch.set_default_dtype(torch.float)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Current Device: {device}")

n_train = 1000
n_test  = 200

nu = 0.025
# Domain and Data
Xi, Xf = 0.0, 2*np.pi
Yi, Yf = Xi, Xf
Ti, Tf = 0.0, 10.0
Nc, N_test     = 100, 100
Nt = 100
BC_pts, IC_pts = 100, 100

residual_data = create_residual_data(Xi, Xf, Yi, Yf, Ti, Tf, Nc, Nt, N_test)
xt_resid      = residual_data[0].to(device)
xt_test       = residual_data[1].to(device) 

ICBC_data = create_ICBC_data(Xi, Xf, Yi, Yf, Ti, Tf, Nt, BC_pts, IC_pts)
IC_xt     = ICBC_data[0].to(device)
BC_xt_bottom = ICBC_data[1].to(device)
BC_xt_top    = ICBC_data[2].to(device)
BC_xt_left   = ICBC_data[3].to(device)
BC_xt_right  = ICBC_data[4].to(device)


K = 4
path = "../data/"
curl_f = np.load(path + fr'Random_NS_trunc_curl_f_{K}_{Nc}.npy').transpose(1, 0, 2)
omega0 = np.load(path + fr"Random_NS_trunc_omega0_{K}_{Nc}.npy").T
omega  = np.load(path + fr"Random_NS_trunc_omega_{K}_{Nc}.npy").transpose(1, 0, 2)

omega0 = torch.from_numpy(omega0.reshape(-1,1).astype(np.float32)).to(device)

curl_f_train = torch.from_numpy(curl_f[:, :, :n_train].astype(np.float32)).to(device)
# omega_train = torch.from_numpy(omega[:, :n_train]).to(device)

curl_f_test = torch.from_numpy(curl_f[:, :, n_train:n_train+n_test].astype(np.float32)).to(device)
# omega_test = torch.from_numpy(omega[:, n_train:n_train+n_test]).to(device)

omega_train = omega[:, :, :n_train]
omega_test  = omega[:, :, n_train:(n_train+n_test)]

number_of_neurons = 20

P_list = np.ones(number_of_neurons, dtype=object)

P_resid_vorstr_values = torch.ones((xt_resid.shape[0], 2, number_of_neurons)).to(device)
P_IC_values    = torch.ones((   IC_xt.shape[0], number_of_neurons)).to(device)
P_BC_bottom    = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_top       = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_left      = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)
P_BC_right     = torch.ones((   BC_xt_bottom.shape[0], 2, number_of_neurons)).to(device)

P_t_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_x_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_xx_term = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_y_term  = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_yy_term = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
P_vel     = torch.ones((xt_resid.shape[0], 2, number_of_neurons)).to(device)
P_lap_psi = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)
Pt_nu_lap_omega = torch.ones((xt_resid.shape[0], number_of_neurons)).to(device)

x_test = xt_test[:, 0:1].to(device)
y_test = xt_test[:, 1:2].to(device)
t_test = xt_test[:, 2:3].to(device)

IC_x = IC_xt[:, 0:1].to(device)
IC_y = IC_xt[:, 1:2].to(device)
IC_t = IC_xt[:, 2:3].to(device)
BC_x_top = BC_xt_top[:, 0:1].to(device)
BC_y_top = BC_xt_top[:, 1:2].to(device)
BC_t_top = BC_xt_top[:, 2:3].to(device)
BC_x_bottom = BC_xt_bottom[:, 0:1].to(device)
BC_y_bottom = BC_xt_bottom[:, 1:2].to(device)
BC_t_bottom = BC_xt_bottom[:, 2:3].to(device)
BC_x_left = BC_xt_left[:, 0:1].to(device)
BC_y_left = BC_xt_left[:, 1:2].to(device)
BC_t_left = BC_xt_left[:, 2:3].to(device)
BC_x_right = BC_xt_right[:, 0:1].to(device)
BC_y_right = BC_xt_right[:, 1:2].to(device)
BC_t_right = BC_xt_right[:, 2:3].to(device)

# gpt_sols_train = torch.ones((N_test, N_test, n_train), dtype = torch.float).to(device)
# gpt_sols_test  = torch.ones((N_test, N_test, n_test), dtype = torch.float).to(device)

layers_pinn = np.array([3, 20, 20, 20, 20, 20, 20, 2])
layers_gpt  = np.array([3, number_of_neurons, 1])

lr_gpt          = 0.005
epochs_gpt      = 5000
epochs_gpt_test = 5000

"""
for i in range(number_of_neurons):
    path = fr"../data/Full-PINN-Data (NS) (K={K})/({i+1})"
    weights = []
    bias = []
    for k in range(len(layers_pinn)-1):
        weights.append(np.loadtxt(fr"{path}/saved_w{k+1}.txt"))
        bias.append(np.loadtxt(fr"{path}/saved_b{k+1}.txt"))
    P_list[i] = P(layers_pinn, weights, bias).to(device)
    
    P_t, P_x, P_xx, P_y, P_yy, vel, lap_psi = autograd_calculations(x_test, y_test, t_test, P_list[i]) 
    Pt_nu_lap_omega[:, i][:, None] = Pt_nu_lap_vor(nu, P_t, P_xx, P_yy)

    P_t_term[:,i][:,None]  = P_t # torch.from_numpy(u_t)
    P_x_term[:,i][:,None]  = P_x # torch.from_numpy(u_x)
    P_xx_term[:,i][:,None] = P_xx # torch.from_numpy(u_xx)
    P_y_term[:,i][:,None]  = P_y 
    P_yy_term[:,i][:,None] = P_yy
    
    
    P_IC_values[:, i][:,None] = P_list[i](IC_x, IC_y, IC_t)[:, 0:1].detach() 
    P_BC_top[:, :, i]    = P_list[i](BC_x_top, BC_y_top, BC_t_top).detach() 
    P_BC_bottom[:, :, i] = P_list[i](BC_x_bottom, BC_y_bottom, BC_t_bottom).detach()
    P_BC_left[:, :, i]   = P_list[i](BC_x_left, BC_y_left, BC_t_left).detach()
    P_BC_right[:, :, i]  = P_list[i](BC_x_right, BC_y_right, BC_t_right).detach()
    
    P_resid_vorstr_values[:, :, i] = P_list[i](x_test, y_test, t_test).detach()
    P_vel[:, :, i] = vel
    P_lap_psi[:, i][:, None] = lap_psi
    
    
    
    
    
losses_train = np.zeros(n_train, dtype = np.float32)
losses_test = np.zeros(n_test, dtype = np.float32)


total_train_time_1 = time.perf_counter()
for i in range(n_train):
    c_initial = torch.full((1, number_of_neurons), 1/number_of_neurons)
    curl_f_gpt_train = curl_f_train[:, :, i].reshape(-1, 1).to(device)
    GPT_NN = GPT(layers_gpt, nu, P_list, c_initial, omega0, curl_f_gpt_train, 
                 P_resid_vorstr_values, P_IC_values, P_BC_bottom, 
                 P_BC_top, P_BC_left, P_BC_right, Pt_nu_lap_omega, P_vel, P_x_term, P_y_term, P_lap_psi).to(device)
    gpt_losses = gpt_train(GPT_NN, nu, xt_resid, IC_xt, BC_xt_bottom, BC_xt_top, BC_xt_left,        
                           BC_xt_right, epochs_gpt, lr_gpt, testing=True)
            
    losses_train[i] = gpt_losses
    
    u_gpt_pred = GPT_NN(testing=True, test_x=x_test, test_y=y_test, test_t=t_test)
    gpt_sols_train[:, :, i] = u_gpt_pred.detach().reshape(N_test, N_test)
total_train_time_2 = time.perf_counter()

print('total train time : ', total_train_time_2 - total_train_time_1, ' seconds')


total_test_time_1 = time.perf_counter()
for i in range(n_test):
    c_initial = torch.full((1, number_of_neurons), 1/number_of_neurons)
    curl_f_gpt_test = curl_f_test[:, :, i].reshape(-1, 1).to(device)
    GPT_NN = GPT(layers_gpt, nu, P_list, c_initial, omega0, curl_f_gpt_test, 
                 P_resid_vorstr_values, P_IC_values, P_BC_bottom, 
                 P_BC_top, P_BC_left, P_BC_right, Pt_nu_lap_omega, P_vel, P_x_term, P_y_term, P_lap_psi).to(device)
    gpt_losses = gpt_train(GPT_NN, nu, xt_resid, IC_xt, BC_xt_bottom, BC_xt_top, BC_xt_left,        
                           BC_xt_right, epochs_gpt, lr_gpt, testing=True)
    
    losses_test[i] = gpt_losses
    
    u_gpt_pred = GPT_NN(testing=True, test_x=x_test, test_y=y_test, test_t=t_test)
    gpt_sols_test[:, :, i] = u_gpt_pred.detach().reshape(N_test, N_test)
total_test_time_2 = time.perf_counter()

print('total test time : ', total_test_time_2 - total_test_time_1, ' seconds')

gpt_sols_train = gpt_sols_train.cpu().numpy()
gpt_sols_test  = gpt_sols_test.cpu().numpy()
"""
path = '../data/'
gpt_sols_train = np.load(path + fr'GPT_PINN_solutions_train_K_{K}_n_{n_train}.npy')
gpt_sols_test  = np.load(path + fr'GPT_PINN_solutions_test_K_{K}_n_{n_test}.npy')
print(gpt_sols_train.shape, gpt_sols_test.shape)

# print(f'Mean train loss: {np.mean(losses_train)}, Mean test loss: {np.mean(losses_test)}')

rel_err_gpt_train = np.zeros(n_train)
rel_err_gpt_test  = np.zeros(n_test)

omega = omega.transpose(1, 0, 2)
curl_f = curl_f.transpose(1, 0, 2)
omega_train = omega_train.transpose(1, 0, 2)
omega_test  = omega_test.transpose(1, 0, 2)

for i in range(n_train):
    rel_err_gpt_train[i] = np.linalg.norm(gpt_sols_train[:, :, i] - omega_train[:, :, i])/np.linalg.norm(omega_train[:, :, i])

for i in range(n_test):
    rel_err_gpt_test[i] = np.linalg.norm(gpt_sols_test[:, :, i] - omega_test[:, :, i])/np.linalg.norm(omega_test[:, :, i])
    
print(f'GPT-PINN: mean rel train err: {np.mean(rel_err_gpt_train)}, mean rel test err: {np.mean(rel_err_gpt_test)}\n')
print(f'GPT-PINN: largest rel train err: {np.max(rel_err_gpt_train)}, largest rel test err: {np.max(rel_err_gpt_test)}')

xgrid = np.linspace(0, 2*np.pi, Nc+1)
xgrid = xgrid[0:-1]
X, Y = np.meshgrid(xgrid, xgrid)

arg_max_train = np.argmax(rel_err_gpt_train)
arg_max_test  = np.argmax(rel_err_gpt_test)

fig, ax = plt.subplots(2, ncols=4, figsize=(10, 4), sharey=True, sharex=True)
im0 = ax[0, 0].pcolormesh(X, Y, curl_f[:, :, arg_max_train], shading='gouraud', cmap='rainbow')
im1 = ax[0, 1].pcolormesh(X, Y, omega[:, :, arg_max_train], shading='gouraud', cmap='rainbow')
im2 = ax[0, 2].pcolormesh(X, Y, gpt_sols_train[:, :, arg_max_train], shading='gouraud', cmap='rainbow')
im3 = ax[0, 3].pcolormesh(X, Y, abs(omega[:, :, arg_max_train] - gpt_sols_train[:, :, arg_max_train]), shading='gouraud', cmap='rainbow', norm=LogNorm())
im4 = ax[1, 0].pcolormesh(X, Y, curl_f[:, :, n_train+arg_max_test], shading='gouraud', cmap='rainbow')
im5 = ax[1, 1].pcolormesh(X, Y, omega[:, :, n_train+arg_max_test], shading='gouraud', cmap='rainbow')
im6 = ax[1, 2].pcolormesh(X, Y, gpt_sols_test[:, :, arg_max_test], shading='gouraud', cmap='rainbow')
im7 = ax[1, 3].pcolormesh(X, Y, abs(omega[:, :, n_train+arg_max_test] - gpt_sols_test[:, :, arg_max_test]), shading='gouraud', cmap='rainbow', norm=LogNorm())
fig.colorbar(im0, ax=ax[0, 0])
fig.colorbar(im1, ax=ax[0, 1])
fig.colorbar(im2, ax=ax[0, 2])
fig.colorbar(im3, ax=ax[0, 3])
fig.colorbar(im4, ax=ax[1, 0])
fig.colorbar(im5, ax=ax[1, 1])
fig.colorbar(im6, ax=ax[1, 2])
fig.colorbar(im7, ax=ax[1, 3])
ax[0, 0].set_title("$\\mathrm{curl}\ f$")
ax[0, 1].set_title("Input $\\omega_0$")
ax[0, 2].set_title("Exact $\\omega(T)$")
ax[0, 3].set_title("Predicted $\\omega$")
ax[0, 0].set_ylabel("Train")
ax[1, 0].set_ylabel("Test")
fig.suptitle(f"max rel train err: {np.max(rel_err_gpt_train):.4f}, max rel test err: {np.max(rel_err_gpt_test):.4f}")
fig.tight_layout()
plt.savefig("../figs/GPT_PINN_largest_rel_err.pdf")
plt.show()

Current Device: cpu


FileNotFoundError: [Errno 2] No such file or directory: '../data/GPT_PINN_solutions_test_K_4.npy'